In [3]:
import pandas as pd
import numpy as np
from datetime import datetime

# Extract

In [4]:
timestamp = datetime.now().strftime('%Y-%m-%d%H%M%S')

df = pd.read_csv('../data/dirty_cafe_sales.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [5]:
df.columns = df.columns.str.strip().str.replace(' ', '_').str.lower()
df.head()

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


# Transform

### Transaction ID

In [6]:
df.dropna(subset=['transaction_id'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   transaction_id    10000 non-null  object
 1   item              9667 non-null   object
 2   quantity          9862 non-null   object
 3   price_per_unit    9821 non-null   object
 4   total_spent       9827 non-null   object
 5   payment_method    7421 non-null   object
 6   location          6735 non-null   object
 7   transaction_date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


### Item, Payment Method, Location

In [7]:
print(df['item'].unique())
print(df['payment_method'].unique())
print(df['location'].unique())

['Coffee' 'Cake' 'Cookie' 'Salad' 'Smoothie' 'UNKNOWN' 'Sandwich' nan
 'ERROR' 'Juice' 'Tea']
['Credit Card' 'Cash' 'UNKNOWN' 'Digital Wallet' 'ERROR' nan]
['Takeaway' 'In-store' 'UNKNOWN' nan 'ERROR']


In [8]:
df['item'] = df['item'].replace(['ERROR', 'UNKNOWN', np.nan], 'Unknown')
df['payment_method'] = df['payment_method'].replace(['ERROR', 'UNKNOWN', np.nan], 'Unknown')
df['location'] = df['location'].replace(['ERROR', 'UNKNOWN', np.nan], 'Unknown')

In [9]:
print(df['item'].unique())
print(df['payment_method'].unique())
print(df['location'].unique())

['Coffee' 'Cake' 'Cookie' 'Salad' 'Smoothie' 'Unknown' 'Sandwich' 'Juice'
 'Tea']
['Credit Card' 'Cash' 'Unknown' 'Digital Wallet']
['Takeaway' 'In-store' 'Unknown']


### Quantity, Price Per Unit, Total Spent

In [10]:
print(df['quantity'].unique())
print(df['price_per_unit'].unique())
print(df['total_spent'].unique())

['2' '4' '5' '3' '1' 'ERROR' 'UNKNOWN' nan]
['2.0' '3.0' '1.0' '5.0' '4.0' '1.5' nan 'ERROR' 'UNKNOWN']
['4.0' '12.0' 'ERROR' '10.0' '20.0' '9.0' '16.0' '15.0' '25.0' '8.0' '5.0'
 '3.0' '6.0' nan 'UNKNOWN' '2.0' '1.0' '7.5' '4.5' '1.5']


In [11]:
df[['quantity', 'price_per_unit', 'total_spent']] = df[['quantity', 'price_per_unit', 'total_spent']].replace(['ERROR', 'UNKNOWN'], np.nan)
df[['quantity', 'price_per_unit', 'total_spent']] = df[['quantity', 'price_per_unit', 'total_spent']].astype(float)

In [12]:
df.loc[df['quantity'].isna() & df['price_per_unit'].notna() & df['total_spent'].notna(), 'quantity'] = df['total_spent'] / df['price_per_unit']
df.loc[df['price_per_unit'].isna() & df['quantity'].notna() & df['total_spent'].notna(), 'price_per_unit'] = df['total_spent'] / df['quantity']
df.loc[df['total_spent'].isna() & df['quantity'].notna() & df['price_per_unit'].notna(), 'total_spent'] = df['quantity'] * df['price_per_unit']

In [13]:
print(df['quantity'].unique())
print(df['price_per_unit'].unique())
print(df['total_spent'].unique())

[ 2.  4.  5.  3.  1. nan]
[2.  3.  1.  5.  4.  1.5 nan]
[ 4.  12.  10.  20.   9.  16.  15.  25.   8.   5.   3.   6.   2.   nan
  1.   7.5  4.5  1.5]


### Transaction Date

In [14]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'], errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    10000 non-null  object        
 1   item              10000 non-null  object        
 2   quantity          9962 non-null   float64       
 3   price_per_unit    9962 non-null   float64       
 4   total_spent       9960 non-null   float64       
 5   payment_method    10000 non-null  object        
 6   location          10000 non-null  object        
 7   transaction_date  9540 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 625.1+ KB


## Drop Row

### Restore `price_per_unit` value based on `item`

In [15]:
price_list = df.dropna(subset=['price_per_unit'])
price_list.groupby('item')['price_per_unit'].apply(lambda x: x.unique())

item
Cake                                 [3.0]
Coffee                               [2.0]
Cookie                               [1.0]
Juice                                [3.0]
Salad                                [5.0]
Sandwich                             [4.0]
Smoothie                             [4.0]
Tea                                  [1.5]
Unknown     [3.0, 1.5, 2.0, 1.0, 5.0, 4.0]
Name: price_per_unit, dtype: object

In [16]:
price_list = df[df['item'].ne('Unknown')]
price_list['item'].unique()

array(['Coffee', 'Cake', 'Cookie', 'Salad', 'Smoothie', 'Sandwich',
       'Juice', 'Tea'], dtype=object)

In [17]:
price_map = (
    price_list
      .groupby('item')['price_per_unit']
      .first()
      .to_dict()
)

print(price_map)

{'Cake': 3.0, 'Coffee': 2.0, 'Cookie': 1.0, 'Juice': 3.0, 'Salad': 5.0, 'Sandwich': 4.0, 'Smoothie': 4.0, 'Tea': 1.5}


In [18]:
df['price_per_unit'] = df['price_per_unit'].fillna(df['item'].map(price_map))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    10000 non-null  object        
 1   item              10000 non-null  object        
 2   quantity          9962 non-null   float64       
 3   price_per_unit    9994 non-null   float64       
 4   total_spent       9960 non-null   float64       
 5   payment_method    10000 non-null  object        
 6   location          10000 non-null  object        
 7   transaction_date  9540 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 625.1+ KB


In [19]:
df.loc[df['quantity'].isna() & df['price_per_unit'].notna() & df['total_spent'].notna(), 'quantity'] = df['total_spent'] / df['price_per_unit']
df.loc[df['price_per_unit'].isna() & df['quantity'].notna() & df['total_spent'].notna(), 'price_per_unit'] = df['total_spent'] / df['quantity']
df.loc[df['total_spent'].isna() & df['quantity'].notna() & df['price_per_unit'].notna(), 'total_spent'] = df['quantity'] * df['price_per_unit']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    10000 non-null  object        
 1   item              10000 non-null  object        
 2   quantity          9977 non-null   float64       
 3   price_per_unit    9994 non-null   float64       
 4   total_spent       9977 non-null   float64       
 5   payment_method    10000 non-null  object        
 6   location          10000 non-null  object        
 7   transaction_date  9540 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 625.1+ KB


### Separate reject and clean data

In [20]:
data_reject = df.loc[df['quantity'].isna() | df['price_per_unit'].isna() | df['total_spent'].isna() | df['transaction_date'].isna()]
data_clean = df.drop(data_reject.index)

In [21]:
data_reject

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
11,TXN_3051279,Sandwich,2.0,4.0,8.0,Credit Card,Takeaway,NaT
29,TXN_7640952,Cake,4.0,3.0,12.0,Digital Wallet,Takeaway,NaT
33,TXN_7710508,Unknown,5.0,1.0,5.0,Cash,Unknown,NaT
77,TXN_2091733,Salad,1.0,5.0,5.0,Unknown,In-store,NaT
103,TXN_7028009,Cake,4.0,3.0,12.0,Unknown,Takeaway,NaT
...,...,...,...,...,...,...,...,...
9933,TXN_9460419,Cake,1.0,3.0,3.0,Unknown,Takeaway,NaT
9937,TXN_8253472,Cake,1.0,3.0,3.0,Unknown,Unknown,NaT
9949,TXN_3130865,Juice,3.0,3.0,9.0,Unknown,In-store,NaT
9983,TXN_9226047,Smoothie,3.0,4.0,12.0,Cash,Unknown,NaT


In [22]:
data_clean

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4.0,1.0,4.0,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2.0,5.0,10.0,Unknown,Unknown,2023-04-27
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-06-11
...,...,...,...,...,...,...,...,...
9995,TXN_7672686,Coffee,2.0,2.0,4.0,Unknown,Unknown,2023-08-30
9996,TXN_9659401,Unknown,3.0,1.0,3.0,Digital Wallet,Unknown,2023-06-02
9997,TXN_5255387,Coffee,4.0,2.0,8.0,Digital Wallet,Unknown,2023-03-02
9998,TXN_7695629,Cookie,3.0,1.0,3.0,Digital Wallet,Unknown,2023-12-02


### `dim_items`

In [23]:
dim_items = data_clean[['item', 'price_per_unit']].drop_duplicates().reset_index(drop=True)
dim_items = dim_items.sort_values(
    by=['item'],
    key=lambda col: col.str.lower().eq('unknown')
).reset_index(drop=True)
dim_items['item_id'] = range(1, len(dim_items) + 1)

dim_items = dim_items[['item_id', 'item', 'price_per_unit']]
dim_items

,item_id,item,price_per_unit
0,1,Coffee,2.0
1,2,Cake,3.0
2,3,Cookie,1.0
3,4,Salad,5.0
4,5,Smoothie,4.0
5,6,Sandwich,4.0
6,7,Juice,3.0
7,8,Tea,1.5
8,9,Unknown,3.0
9,10,Unknown,1.5


### `dim_payment_methods`

In [24]:
dim_payment_methods = data_clean[['payment_method']].drop_duplicates().reset_index(drop=True)
dim_payment_methods = dim_payment_methods.sort_values(
    by=['payment_method'],
    key=lambda col: col.str.lower().eq('unknown')
).reset_index(drop=True)
dim_payment_methods['payment_method_id'] = range(1, len(dim_payment_methods) + 1)

dim_payment_methods = dim_payment_methods[['payment_method_id', 'payment_method']]
dim_payment_methods

,payment_method_id,payment_method
0,1,Credit Card
1,2,Cash
2,3,Digital Wallet
3,4,Unknown


### `dim_locations`

In [25]:
dim_locations = data_clean[['location']].drop_duplicates().reset_index(drop=True)
dim_locations = dim_locations.sort_values(
    by=['location'],
    key=lambda col: col.str.lower().eq('unknown')
).reset_index(drop=True)
dim_locations['location_id'] = range(1, len(dim_locations) + 1)

dim_locations = dim_locations[['location_id', 'location']]
dim_locations

,location_id,location
0,1,Takeaway
1,2,In-store
2,3,Unknown


### `fact_transactions`

In [26]:
fact_transactions = data_clean.merge(dim_items, on=['item', 'price_per_unit'], how='left') \
    .merge(dim_payment_methods, on='payment_method', how='left') \
    .merge(dim_locations, on='location', how='left')

fact_transactions = fact_transactions[[
    'transaction_id',
    'transaction_date',
    'item_id',
    'payment_method_id',
    'location_id',
    'quantity',
    'total_spent'
]]

fact_transactions

,transaction_id,transaction_date,item_id,payment_method_id,location_id,quantity,total_spent
0,TXN_1961373,2023-09-08,1,1,1,2.0,4.0
1,TXN_4977031,2023-05-16,2,2,2,4.0,12.0
2,TXN_4271903,2023-07-19,3,1,2,4.0,4.0
3,TXN_7034554,2023-04-27,4,4,3,2.0,10.0
4,TXN_3160411,2023-06-11,1,3,2,2.0,4.0
...,...,...,...,...,...,...,...
9509,TXN_7672686,2023-08-30,1,4,3,2.0,4.0
9510,TXN_9659401,2023-06-02,12,3,3,3.0,3.0
9511,TXN_5255387,2023-03-02,1,3,3,4.0,8.0
9512,TXN_7695629,2023-12-02,3,3,3,3.0,3.0


# Load

### Data Reject

In [27]:
data_reject.to_csv(f'../target/reject/data_rejected_{timestamp}.csv', index=False)

### Data Accepted

In [28]:
dim_items.to_csv(f'../target/accepted/dim_items_{timestamp}.csv', index=False)
dim_payment_methods.to_csv(f'../target/accepted/dim_payment_methods_{timestamp}.csv', index=False)
dim_locations.to_csv(f'../target/accepted/dim_locations_{timestamp}.csv', index=False)
fact_transactions.to_csv(f'../target/accepted/fact_transactions_{timestamp}.csv', index=False)